In [1]:
!pip install 'zarr<3'
!pip install timm
!pip install openslide-python tiffslide

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## Step 2: Import VitaminP modules

In [3]:
# Cell 2: Import VitaminP
from vitaminp import VitaminPFlex, VitaminPDual
from vitaminp.inference import WSIPredictor
from vitaminp.inference.wsi_handler import WSIHandler
from vitaminp.inference.utils import setup_logger

print("✓ VitaminP imports successful!")
print(f"  - VitaminPFlex: {VitaminPFlex}")
print(f"  - VitaminPDual: {VitaminPDual}")
print(f"  - WSIPredictor: {WSIPredictor}")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ VitaminP imports successful!
  - VitaminPFlex: <class 'vitaminp.models.VitaminPFlex'>
  - VitaminPDual: <class 'vitaminp.models.VitaminPDual'>
  - WSIPredictor: <class 'vitaminp.inference.predictor.WSIPredictor'>


## Step 3: Initialize Model

In [4]:
import torch
# Cell 3: Initialize VitaminP Model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize your model (use the same one you trained)
model = VitaminPFlex(
    model_size='large',  # Change to your model size: 'small', 'base', 'large', 'xlarge'
    freeze_backbone=False
)

print(f"✓ Model initialized: {model.__class__.__name__}")
print(f"  Model size: large")
print(f"  Device: {device}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")

✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]
✓ Model initialized: VitaminPFlex
  Model size: large
  Device: cuda
  Total parameters: 469,380,236


## Step 4: Create WSIPredictor

In [5]:
# Cell 4: Create WSIPredictor (UPDATED)
checkpoint_path = 'checkpoints/vitamin_p_flex_large_fold2_best.pth'
output_dir = 'results/test_inference'

predictor = WSIPredictor(
    model=model,
    checkpoint_path=checkpoint_path,
    device=device,
    batch_size=None,
    patch_size=512,  # CHANGED from 1024 to 512
    overlap=64,
    target_mpp=0.3225,
    magnification=40,
    mixed_precision=True,
    num_workers=4,
)

print("✓ WSIPredictor created successfully!")
print(predictor)

2025-12-24 16:53:02 - vitaminp.inference - INFO - Loading checkpoint: checkpoints/vitamin_p_flex_large_fold2_best.pth
2025-12-24 16:53:03 - vitaminp.inference - INFO - Checkpoint loaded successfully
2025-12-24 16:53:03 - vitaminp.inference.tile_processor - INFO - GPU memory: 84.93 GB
2025-12-24 16:53:03 - vitaminp.inference.tile_processor - INFO - Estimated batch size: 25
2025-12-24 16:53:03 - vitaminp.inference - INFO - WSIPredictor initialized: device=cuda, batch_size=25, patch_size=512, overlap=64, target_mpp=0.3225, magnification=40


✓ WSIPredictor created successfully!
WSIPredictor(
  model=VitaminPFlex,
  checkpoint=vitamin_p_flex_large_fold2_best.pth,
  device=cuda,
  batch_size=25,
  patch_size=512,
  overlap=64,
  magnification=40
)


## Step 6: Run Full Inference!

In [6]:
# RECREATE the predictor with correct patch_size
from vitaminp import VitaminPFlex
from vitaminp.inference import WSIPredictor

# Setup model
device = 'cuda'
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold2_best.pth", map_location=device))
model.eval()

# ✅ CREATE NEW PREDICTOR with patch_size=512
predictor = WSIPredictor(
    model=model,
    checkpoint_path='checkpoints/vitamin_p_flex_large_fold2_best.pth',
    device=device,
    patch_size=512,      # ✅ MUST BE 512
    overlap=64,
    target_mpp=0.25,
    magnification=40,
    mixed_precision=True,
)

print(f"Predictor config: patch_size={predictor.patch_size}, overlap={predictor.overlap}")

# Now run inference with the NEW predictor
test_wsi_path = '/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/results/test_region_5000x5000.png'
output_dir = 'results/full_inference_512'  # New output directory

wsi_properties = {
    'slide_mpp': 0.325,
    'magnification': 40
}

results = predictor.predict(
    wsi_path=test_wsi_path,
    output_dir=output_dir,
    branch='he_nuclei',
    wsi_properties=wsi_properties,
    filter_tissue=True,
    tissue_threshold=0.1,
    clean_overlaps=True,
    iou_threshold=0.5,
    save_json=True,
    save_csv=True,
    save_geojson=True,
    save_heatmap=True,
)

print("\n" + "="*60)
print("INFERENCE COMPLETE!")
print("="*60)
print(f"Branch: {results['branch']}")
print(f"Total detections: {results['num_detections']}")
print(f"Processing time: {results['processing_time']:.2f} seconds")
print(f"Results saved to: {output_dir}")


✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


2025-12-24 16:53:09 - vitaminp.inference - INFO - Loading checkpoint: checkpoints/vitamin_p_flex_large_fold2_best.pth
2025-12-24 16:53:10 - vitaminp.inference - INFO - Checkpoint loaded successfully
2025-12-24 16:53:10 - vitaminp.inference.tile_processor - INFO - GPU memory: 84.93 GB
2025-12-24 16:53:10 - vitaminp.inference.tile_processor - INFO - Estimated batch size: 25
2025-12-24 16:53:10 - vitaminp.inference - INFO - WSIPredictor initialized: device=cuda, batch_size=25, patch_size=512, overlap=64, target_mpp=0.25, magnification=40
2025-12-24 16:53:10 - vitaminp.inference - INFO - Starting inference on WSI: test_region_5000x5000.png
2025-12-24 16:53:10 - vitaminp.inference - INFO - Using branch: he_nuclei
2025-12-24 16:53:10 - vitaminp.inference - INFO - Step 1/5: Loading WSI and generating tiles...
2025-12-24 16:53:10 - vitaminp.inference - INFO - Loading WSI: test_region_5000x5000.png
2025-12-24 16:53:10 - vitaminp.inference - INFO - Loading as simple image (PNG/JPEG/etc.)...


Predictor config: patch_size=512, overlap=64


2025-12-24 16:53:11 - vitaminp.inference - INFO - WSI loaded successfully using simple_image
2025-12-24 16:53:11 - vitaminp.inference - INFO - Using provided slide_mpp: 0.325
2025-12-24 16:53:11 - vitaminp.inference - INFO - Using provided magnification: 40
2025-12-24 16:53:11 - vitaminp.inference - WARNING - Slide MPP (0.325) outside optimal range. Using requested target_mpp: 0.25. Handle with care!
2025-12-24 16:53:11 - vitaminp.inference - INFO - Generating tiles for WSI: 3846x3846 (downsampling: 0.769)
2025-12-24 16:53:11 - vitaminp.inference - INFO - Generated 81 tiles
2025-12-24 16:53:11 - vitaminp.inference - INFO - WSI loaded successfully. Total tiles: 81
2025-12-24 16:53:11 - vitaminp.inference - INFO - Step 2/5: Processing tiles through model...
2025-12-24 16:53:11 - vitaminp.inference - INFO - TileProcessor initialized: device=cuda, batch_size=25, mixed_precision=True
2025-12-24 16:53:11 - vitaminp.inference.tile_processor - INFO - Applying tissue filtering to tiles...
Tissu


INFERENCE COMPLETE!
Branch: he_nuclei
Total detections: 0
Processing time: 4.73 seconds
Results saved to: results/full_inference_512


## Sample selections

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Properly extract a region from WSI for testing
Saves as standard TIFF that works with both OpenSlide and TiffSlide
"""

import numpy as np
from PIL import Image
from pathlib import Path
import tifffile

def extract_wsi_region_proper(
    wsi_path,
    output_path,
    x=0,
    y=0,
    width=5000,
    height=5000,
    level=0,
    save_as_ome=False
):
    """
    Properly extract a region from WSI and save as TIFF
    
    Args:
        wsi_path: Path to original WSI
        output_path: Where to save the extracted region
        x, y: Top-left coordinates
        width, height: Region size
        level: Pyramid level (0 = highest resolution)
        save_as_ome: If True, saves as OME-TIFF; otherwise standard TIFF
    """
    wsi_path = Path(wsi_path)
    output_path = Path(output_path)
    
    print(f"Loading WSI: {wsi_path.name}")
    
    # Try to load with available libraries
    try:
        from tiffslide import TiffSlide
        slide = TiffSlide(str(wsi_path))
        reader = "TiffSlide"
        print(f"Loaded with TiffSlide")
    except:
        try:
            from openslide import OpenSlide
            slide = OpenSlide(str(wsi_path))
            reader = "OpenSlide"
            print(f"Loaded with OpenSlide")
        except Exception as e:
            print(f"Error: Could not load WSI with any reader: {e}")
            return None
    
    print(f"WSI dimensions: {slide.dimensions}")
    print(f"WSI levels: {slide.level_count}")
    print(f"Extracting region: x={x}, y={y}, width={width}, height={height}, level={level}")
    
    # Extract region
    try:
        region = slide.read_region(
            location=(x, y),
            level=level,
            size=(width, height)
        )
        print(f"Region extracted: {region.size}, mode={region.mode}")
        
        # Convert RGBA to RGB if needed
        if region.mode == 'RGBA':
            print("Converting RGBA to RGB...")
            region = region.convert('RGB')
        
        # Convert to numpy array
        region_array = np.array(region)
        print(f"Array shape: {region_array.shape}, dtype: {region_array.dtype}")
        
    except Exception as e:
        print(f"Error extracting region: {e}")
        slide.close()
        return None
    
    # Save the region
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    if save_as_ome:
        # Save as OME-TIFF (proper format)
        print(f"Saving as OME-TIFF: {output_path}")
        tifffile.imwrite(
            str(output_path),
            region_array,
            photometric='rgb',
            compression='deflate',  # Use deflate/zip instead of JPEG
            metadata={'axes': 'YXC'}
        )
    else:
        # Save as standard TIFF using PIL (most compatible)
        print(f"Saving as standard TIFF: {output_path}")
        region.save(
            str(output_path),
            'TIFF',
            compression='tiff_deflate'  # Use deflate instead of JPEG
        )
    
    slide.close()
    
    print(f"✓ Saved successfully: {output_path}")
    print(f"  File size: {output_path.stat().st_size / 1024 / 1024:.1f} MB")
    
    return output_path


def verify_extracted_file(file_path):
    """Verify the extracted file can be opened"""
    print(f"\nVerifying file: {file_path.name}")
    
    # Try with PIL
    try:
        img = Image.open(str(file_path))
        print(f"✓ PIL can open: size={img.size}, mode={img.mode}")
    except Exception as e:
        print(f"✗ PIL failed: {e}")
    
    # Try with tifffile
    try:
        img_array = tifffile.imread(str(file_path))
        print(f"✓ tifffile can read: shape={img_array.shape}, dtype={img_array.dtype}")
    except Exception as e:
        print(f"✗ tifffile failed: {e}")
    
    # Try with TiffSlide
    try:
        from tiffslide import TiffSlide
        slide = TiffSlide(str(file_path))
        print(f"✓ TiffSlide can open: dimensions={slide.dimensions}, levels={slide.level_count}")
        slide.close()
    except Exception as e:
        print(f"✗ TiffSlide failed: {e}")
    
    # Try with OpenSlide
    try:
        from openslide import OpenSlide
        slide = OpenSlide(str(file_path))
        print(f"✓ OpenSlide can open: dimensions={slide.dimensions}, levels={slide.level_count}")
        slide.close()
    except Exception as e:
        print(f"✗ OpenSlide failed: {e}")


if __name__ == "__main__":
    # Configuration
    original_wsi = '/rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/H&Es/MS004_HE.svs'
    output_dir = Path('/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images')

    print("="*60)
    print("WSI Region Extractor (Proper Version)")
    print("="*60)
    
    # Extract as standard TIFF (most compatible)
    output_file = output_dir / 'svs4.tif'
    extracted = extract_wsi_region_proper(
        wsi_path=original_wsi,
        output_path=output_file,
        x=10000,  # Adjust these coordinates as needed
        y=5000,
        width=2000,
        height=2000,
        level=0,
        save_as_ome=False  # Use False for standard TIFF (more compatible)
    )
    
    if extracted:
        print("\n" + "="*60)
        print("Verification")
        print("="*60)
        verify_extracted_file(extracted)
        
        print("\n" + "="*60)
        print("SUCCESS!")
        print("="*60)
        print(f"Use this file in your inference:")
        print(f"test_wsi_path = '{extracted}'")

WSI Region Extractor (Proper Version)
Loading WSI: MS004_HE.svs
Loaded with OpenSlide
WSI dimensions: (21912, 12917)
WSI levels: 3
Extracting region: x=10000, y=5000, width=2000, height=2000, level=0
Region extracted: (2000, 2000), mode=RGBA
Converting RGBA to RGB...
Array shape: (2000, 2000, 3), dtype: uint8
Saving as standard TIFF: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/svs4.tif
✓ Saved successfully: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/svs4.tif
  File size: 10.2 MB

Verification

Verifying file: svs4.tif
✓ PIL can open: size=(2000, 2000), mode=RGB
✓ tifffile can read: shape=(2000, 2000, 3), dtype=uint8
✗ TiffSlide failed: No module named 'tiffslide'
✗ OpenSlide failed: Unsupported or missing image file

SUCCESS!
Use this file in your inference:
test_wsi_path = '/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images/svs4.tif'


 ## mIF extractions

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Extract MIF region for VitaminP testing
"""

import tifffile
import numpy as np
from pathlib import Path
from PIL import Image

def extract_mif_region(
    mif_path,
    output_dir,
    region_x=10000,
    region_y=10000,
    region_width=3000,
    region_height=3000,
    save_individual_channels=True,
    create_composite_rgb=True
):
    """
    Extract a region from MIF OME-TIFF
    
    Args:
        mif_path: Path to MIF OME-TIFF
        output_dir: Output directory
        region_x, region_y: Top-left corner
        region_width, region_height: Region size
        save_individual_channels: Save each channel as separate image
        create_composite_rgb: Create RGB composite for visualization
    
    Returns:
        Dictionary with paths to saved files
    """
    mif_path = Path(mif_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print("="*60)
    print(f"Extracting MIF Region: {mif_path.name}")
    print("="*60)
    
    # Channel names
    channel_names = [
        'FITC_525nm_SYTO13',
        'Cy3_568nm_Alexa532', 
        'TexasRed_615nm_Alexa594',
        'Cy5_666nm'
    ]
    
    with tifffile.TiffFile(str(mif_path)) as tif:
        # Load full array
        print(f"\n📂 Loading image...")
        full_array = tif.asarray()
        print(f"  Full shape: {full_array.shape}")
        
        # Extract region
        print(f"\n✂️  Extracting region:")
        print(f"  Position: ({region_x}, {region_y})")
        print(f"  Size: {region_width} × {region_height}")
        
        region = full_array[
            :,
            region_y:region_y+region_height,
            region_x:region_x+region_width
        ]
        
        print(f"  Extracted shape: {region.shape}")
        print(f"  Dtype: {region.dtype}")
    
    # Save as OME-TIFF (for VitaminP MIF branch)
    output_ome = output_dir / f'mif_region.ome.tiff'
    print(f"\n💾 Saving OME-TIFF: {output_ome.name}")
    tifffile.imwrite(
        str(output_ome),
        region,
        photometric='minisblack',
        metadata={'axes': 'CYX'}
    )
    print(f"  ✓ Saved: {output_ome}")
    print(f"  File size: {output_ome.stat().st_size / 1024 / 1024:.1f} MB")
    
    saved_files = {'ome_tiff': output_ome}
    
    # Save individual channel previews
    if save_individual_channels:
        print(f"\n🖼️  Saving individual channel previews...")
        for i, ch_name in enumerate(channel_names):
            ch_data = region[i]
            
            # Normalize to 8-bit (enhance contrast)
            p1, p99 = np.percentile(ch_data[ch_data > 0], [1, 99])
            ch_norm = np.clip((ch_data - p1) / (p99 - p1) * 255, 0, 255).astype(np.uint8)
            
            # Save as PNG
            preview_path = output_dir / f'mif_region_CH{i}_{ch_name}.png'
            Image.fromarray(ch_norm).save(preview_path)
            print(f"  ✓ Channel {i} ({ch_name}): {preview_path.name}")
            saved_files[f'channel_{i}'] = preview_path
    
    # Create RGB composite (for visualization only)
    if create_composite_rgb:
        print(f"\n🎨 Creating RGB composite...")
        
        # Map channels to RGB
        # FITC (green) -> G channel
        # Cy3 (orange) -> R+G channels
        # Texas Red (red) -> R channel
        # Cy5 (far red) -> R channel
        
        # Normalize each channel
        def normalize_channel(data):
            if data.max() > 0:
                p1, p99 = np.percentile(data[data > 0], [1, 99])
                return np.clip((data - p1) / (p99 - p1) * 255, 0, 255).astype(np.uint8)
            return np.zeros_like(data, dtype=np.uint8)
        
        r_channel = normalize_channel(region[2] + region[3])  # Texas Red + Cy5
        g_channel = normalize_channel(region[0])  # FITC
        b_channel = np.zeros_like(r_channel)  # No blue fluorophore
        
        rgb_composite = np.stack([r_channel, g_channel, b_channel], axis=-1)
        
        composite_path = output_dir / 'mif_region_composite_RGB.png'
        Image.fromarray(rgb_composite).save(composite_path)
        print(f"  ✓ RGB composite: {composite_path.name}")
        saved_files['rgb_composite'] = composite_path
    
    print(f"\n" + "="*60)
    print("✅ Extraction Complete!")
    print("="*60)
    
    return saved_files


if __name__ == "__main__":
    mif_path = '/rsrch9/home/plm/idso_fa1_pathology/TIER1/patient-mosaic/2014-0938/MOSAIC/DSP/MOSAIC DSP Set 1/Images/DSP Scans/MS001S1.png'
    output_dir = Path('/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest/test_images')
    
    # Extract a region
    files = extract_mif_region(
        mif_path=mif_path,
        output_dir=output_dir,
        region_x=10000,      # Adjust these coordinates
        region_y=10000,      # to find a tissue-rich region
        region_width=3000,
        region_height=3000,
        save_individual_channels=True,
        create_composite_rgb=True
    )
    
    print(f"\n📋 Saved Files:")
    for key, path in files.items():
        print(f"  {key}: {path}")
    
    print(f"\n🚀 Next: Run VitaminP inference on MIF:")
    print(f"""
results = predictor.predict(
    wsi_path='{files['ome_tiff']}',
    output_dir='results_mif',
    branch='mif_nuclei',  # or 'mif_cell'
    filter_tissue=True,
    save_geojson=True,
)
""")

Extracting MIF Region: MS001S1.png


TiffFileError: not a TIFF file b'\x89PNG'